<a href="https://colab.research.google.com/github/mohamed97-ba/COVID-19-Tweet-Classification/blob/main/Final_solution_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/My Drive/updated_train.csv')
test_df=pd.read_csv('/content/drive/My Drive/updated_test.csv')
sub=pd.read_csv('/content/drive/My Drive/updated_ss.csv')
feat_cols = "text"
label_cols = "target"


In [ ]:
test_df[feat_cols] = test_df[feat_cols].astype(str)
train[feat_cols] = train[feat_cols].astype(str)

In [ ]:
! pip install pytorch-transformers

     |████████████████████████████████| 184kB 10.1MB/s 
     |████████████████████████████████| 1.0MB 20.1MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=fb9260c5062937346dbf8eca38687175b005454c8bb86bdd90a0162b5f517bb1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from fastai.text import *
from fastai.metrics import *
from pytorch_transformers import RobertaTokenizer

In [ ]:
# Creating a config object to store task specific information
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=False,All Competitions
    seed = 2019,
    roberta_model_name='roberta-large', # can also be exchnaged with roberta-large 
    max_lr=2e-5,
    epochs=2,
    use_fp16=False,
    bs=4, 
    max_seq_len=64, 
    num_labels = 3,
    hidden_dropout_prob=.05,
    hidden_size=1024, # 1024 for roberta-large
    start_tok = "<s>",
    end_tok = "</s>",
)

In [ ]:
train.dtypes

ID        object
text      object
target     int64
dtype: object

In [ ]:
class FastAiRobertaTokenizer(BaseTokenizer):
    """Wrapper around RobertaTokenizer to be compatible with fastai"""
    def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=64, **kwargs): 
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
        return self 
    def tokenizer(self, t:str) -> List[str]: 
        """Adds Roberta bos and eos tokens and limits the maximum sequence length""" 
        return [config.start_tok] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + [config.end_tok]

In [ ]:
# create fastai tokenizer for roberta
roberta_tok = RobertaTokenizer.from_pretrained("roberta-large")

fastai_tokenizer = Tokenizer(tok_func=FastAiRobertaTokenizer(roberta_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

100%|██████████| 456318/456318 [00:00<00:00, 1070872.05B/s]


In [ ]:
# create fastai vocabulary for roberta
path = Path()
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)
    
fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

In [ ]:
# Setting up pre-processors
class RobertaTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
         super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class RobertaNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=fastai_roberta_vocab, **kwargs)


def get_roberta_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for Roberta
    We remove sos and eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original Roberta model.
    """
    return [RobertaTokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(vocab=vocab)]


In [ ]:
# Creating a Roberta specific DataBunch class
class RobertaDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training Roberta"
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
               pad_first=True, device:torch.device=None, no_check:bool=False, backwards:bool=False, 
               dl_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)


In [ ]:
class RobertaTextList(TextList):
    _bunch = RobertaDataBunch
    _label_cls = TextList

In [ ]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=252) \
    .label_from_df(cols=label_cols,label_cls=CategoryList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)

import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=2):
        super(CustomRobertaModel,self).__init__()
        self.drop_out = nn.Dropout(0.1)

        
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits

roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[accuracy])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(1, max_lr=config.max_lr)
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=config.max_lr)

100%|██████████| 1425941629/1425941629 [00:47<00:00, 29819402.34B/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.240777,0.200110,0.917692,04:35


epoch,train_loss,valid_loss,accuracy,time
0,0.275695,0.239441,0.912961,04:42
1,0.129709,0.208319,0.923368,04:42


In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values


test_preds,preds = get_preds_as_nparray(DatasetType.Test)
df=pd.DataFrame(data=test_preds,columns=['a','b'])
sub['target']=df['b']
sub.to_csv('tweet_win1.csv',index=False)

In [ ]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=42) \
    .label_from_df(cols=label_cols,label_cls=CategoryList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)

import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=2):
        super(CustomRobertaModel,self).__init__()
        self.drop_out = nn.Dropout(0.1)

        
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits

roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[accuracy])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(1, max_lr=config.max_lr)
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=config.max_lr)

def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values


test_preds,preds = get_preds_as_nparray(DatasetType.Test)
df=pd.DataFrame(data=test_preds,columns=['a','b'])
sub['target']=df['b']

sub.to_csv('tweet_win2.csv',index=False)

epoch,train_loss,valid_loss,accuracy,time
0,0.176821,0.211046,0.911069,04:37


epoch,train_loss,valid_loss,accuracy,time
0,0.257053,0.212229,0.903500,04:36
1,0.099753,0.198745,0.926206,04:40


In [ ]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=254) \
    .label_from_df(cols=label_cols,label_cls=CategoryList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)

import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=2):
        super(CustomRobertaModel,self).__init__()
        self.drop_out = nn.Dropout(0.1)

        
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits

roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[accuracy])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(1, max_lr=config.max_lr)
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=config.max_lr)

def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values


test_preds,preds = get_preds_as_nparray(DatasetType.Test)
df=pd.DataFrame(data=test_preds,columns=['a','b'])
sub['target']=df['b']

sub.to_csv('tweet_win3.csv',index=False)

epoch,train_loss,valid_loss,accuracy,time
0,0.192366,0.211875,0.919584,04:40


epoch,train_loss,valid_loss,accuracy,time
0,0.287414,0.221960,0.900662,04:47
1,0.129344,0.206682,0.919584,04:43


simple transformers


In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 645kB 7.6MB/s 
     |████████████████████████████████| 3.8MB 24.6MB/s 
     |████████████████████████████████| 194kB 6.0MB/s 
     |████████████████████████████████| 204kB 10.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=8217daccd78ec5d915fc494dfadb827ed07553e7235421618e8aeb373ad54cfd
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [ ]:
import gc
import sklearn
from sklearn.metrics import log_loss
import re
import random
import torch
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold


In [ ]:
train=pd.read_csv('/content/drive/My Drive/updated_train.csv')
test=pd.read_csv('/content/drive/My Drive/updated_test.csv')

train1=train.drop(['ID'],axis=1)
test1=test.drop(['ID'],axis=1)
test1['label']=0

In [ ]:
y_pred_test=[]
fold=StratifiedKFold(n_splits=20, shuffle=True, random_state=2)
i=1
for train_index, test_index in fold.split(train1,train1['target']):
    train1_trn, train1_val = train1.iloc[train_index], train1.iloc[test_index]
    model = ClassificationModel('roberta', 'roberta-large', use_cuda=True,num_labels=2, args={'train_batch_size':32,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 2,
                                                                         'max_seq_length': 64,
                                                                         'regression': False,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":5e-5,
                                                                         'weight_decay':0,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": True})
    model.train_model(train1_trn)
    outputs_test = model.eval_model(test1)[1]
    outputs_test = softmax(outputs_test,axis=1)[:,1]
    y_pred_test.append(outputs_test)
sub=pd.DataFrame()
sub['ID']=test['ID']
sub['target']=np.mean(y_pred_test, 0)
sub.to_csv('simple_transformers2.csv',index=False)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:251: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:668: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  

In [ ]:
import pandas as pd
sub1=pd.read_csv('/content/tweet_win1.csv')
sub3=pd.read_csv('/content/tweet_win2.csv')
sub4=pd.read_csv('/content/tweet_win3.csv')

l=['target']
for col in l:
  sub1[col]=0.35*sub1[col]+0.3*sub3[col]+0.35*sub4[col]

sub1.to_csv('ensemblez1.csv',index=False)
sub1.head()

,ID,target
0,test_2,0.988479
1,test_3,0.000824
2,test_4,0.001790
3,test_8,0.984627
4,test_9,0.002324


**Final Submission**

In [ ]:
import pandas as pd

sub3=pd.read_csv('/content/simple_transformers2.csv')
sub4=pd.read_csv('ensemblez1.csv')

l=['target']
for col in l:
  sub1[col]=0.6*sub3[col]+0.4*sub4[col]

sub1.to_csv('ens4.csv',index=False)#0.1656

sub1.head()

,ID,target
0,test_2,0.962011
1,test_3,0.001105
2,test_4,0.002575
3,test_8,0.985296
4,test_9,0.002277
